In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, GlobalAveragePooling1D, Flatten
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
train_sentences = []
train_labels = []
with open("./dataset/train.txt") as f:
    for items in f:
        item = items.split(";")
        train_sentences.append(item[0])
        train_labels.append(item[1].rstrip())

test_sentences = []
test_labels = []
with open("./dataset/test.txt") as f:
    for items in f:
        item = items.split(";")
        test_sentences.append(item[0])
        test_labels.append(item[1].rstrip())

val_sentences = []
val_labels = []
with open("./dataset/val.txt") as f:
    for items in f:
        item = items.split(";")
        val_sentences.append(item[0])
        val_labels.append(item[1].rstrip())

In [3]:
print(set(train_labels))

{'joy', 'love', 'fear', 'anger', 'surprise', 'sadness'}


In [4]:
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000
vocab_size = 5000
max_length = 32
embedding_dim = 64

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index


train_seq = tokenizer.texts_to_sequences(train_sentences)

train_padded = pad_sequences(train_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)




In [5]:
# tokenizer.fit_on_texts(val_sentences)
val_seq = tokenizer.texts_to_sequences(val_sentences)
val_padded = pad_sequences(val_seq, maxlen=max_length, padding=padding_type, truncating=trunc_type)
val_padded = np.array(val_padded)

print(len(val_seq))
print(val_padded.shape)

2000
(2000, 32)


In [6]:
# label
labels = list(train_labels)
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

train_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
val_label_seq = np.array(label_tokenizer.texts_to_sequences(val_labels))

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 64)          320000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 7)                 455       
                                                                 
Total params: 394,759
Trainable params: 394,759
Non-trainable params: 0
_________________________________________________________________


In [8]:
# from sklearn.preprocessing import OneHotEncoder
# encode = OneHotEncoder()

# onehot_train_label = encode.fit_transform(train_label_seq)
# onehot_val_label = encode.transform(val_label_seq)

In [35]:
onehot_train_label = np.squeeze(tf.one_hot(train_label_seq,7).numpy())
onehot_val_label = np.squeeze(tf.one_hot(val_label_seq,7).numpy())

In [36]:
print(onehot_train_label)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


In [39]:
model.compile(optimizer = 'Adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [40]:
num_epochs = 10

history = model.fit(train_padded, onehot_train_label, epochs=num_epochs, validation_data=(val_padded, onehot_val_label), verbose=1)

Epoch 1/10
500/500 [==============================] - 15s 25ms/step - loss: 1.2052 - accuracy: 0.5314 - val_loss: 0.5736 - val_accuracy: 0.8060
Epoch 2/10
500/500 [==============================] - 14s 28ms/step - loss: 0.3244 - accuracy: 0.8873 - val_loss: 0.2887 - val_accuracy: 0.8925
Epoch 3/10
500/500 [==============================] - 15s 30ms/step - loss: 0.1664 - accuracy: 0.9383 - val_loss: 0.3020 - val_accuracy: 0.8995
Epoch 4/10
500/500 [==============================] - 14s 29ms/step - loss: 0.1232 - accuracy: 0.9547 - val_loss: 0.2972 - val_accuracy: 0.8995
Epoch 5/10
500/500 [==============================] - 14s 28ms/step - loss: 0.0943 - accuracy: 0.9646 - val_loss: 0.3107 - val_accuracy: 0.8975
Epoch 6/10
500/500 [==============================] - 15s 29ms/step - loss: 0.0800 - accuracy: 0.9701 - val_loss: 0.3388 - val_accuracy: 0.8960
Epoch 7/10
500/500 [==============================] - 16s 32ms/step - loss: 0.0682 - accuracy: 0.9741 - val_loss: 0.3495 - val_accuracy:

In [44]:
predicted = model.predict(np.array(['i feel good']))

ValueError: in user code:

    File "d:\Anaconda3\lib\site-packages\keras\engine\training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "d:\Anaconda3\lib\site-packages\keras\engine\training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "d:\Anaconda3\lib\site-packages\keras\engine\training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "d:\Anaconda3\lib\site-packages\keras\engine\training.py", line 1572, in predict_step
        return self(x, training=False)
    File "d:\Anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "d:\Anaconda3\lib\site-packages\keras\engine\input_spec.py", line 213, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" '

    ValueError: Exception encountered when c

In [ ]:
print(val_padded[0])

In [32]:
import time

saved_model_path = './saved_models/{}'.format(int(time.time()))

tf.keras.models.save_model(model, saved_model_path)

INFO:tensorflow:Assets written to: ./saved_models/1653681532\assets


INFO:tensorflow:Assets written to: ./saved_models/1653681532\assets


In [25]:
saved_model_path = "./saved_models/models.h5"

model.save(saved_model_path)

In [28]:
from tensorflow.keras.models import load_model

savedModel=load_model('./saved_models/models.h5')
savedModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 394,694
Trainable params: 394,694
Non-trainable params: 0
_________________________________________________________________


In [33]:
!tensorflowjs_converter --input_format=keras_saved_model \
    ./saved_models/1653681532 \
        ./saved_models/linear

2022-05-28 03:02:11.208731: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-05-28 03:02:11.209145: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-05-28 03:02:26.916560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-05-28 03:02:26.916873: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-28 03:02:26.962074: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: MIHOK
2022-05-28 03:02:26.962456: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: MIHOK
2022-05-28 03:02:26.997326: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneA

In [29]:
json_string = model.to_json()


In [31]:
print(json_string)

{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, null], "dtype": "float32", "sparse": false, "ragged": false, "name": "embedding_input"}}, {"class_name": "Embedding", "config": {"name": "embedding", "trainable": true, "batch_input_shape": [null, null], "dtype": "float32", "input_dim": 5000, "output_dim": 64, "embeddings_initializer": {"class_name": "RandomUniform", "config": {"minval": -0.05, "maxval": 0.05, "seed": null}}, "embeddings_regularizer": null, "activity_regularizer": null, "embeddings_constraint": null, "mask_zero": false, "input_length": null}}, {"class_name": "Bidirectional", "config": {"name": "bidirectional", "trainable": true, "dtype": "float32", "layer": {"class_name": "LSTM", "config": {"name": "lstm", "trainable": true, "dtype": "float32", "return_sequences": false, "return_state": false, "go_backwards": false, "stateful": false, "unroll": false, "time_major": false, "units":